In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2025-05-24 20:25:44.423981: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748118344.704307      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748118344.780411      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/kaggle/input/titanic-test/titanic_test.csv
/kaggle/input/titanic-train/titanic_train.csv


In [2]:
titanic_train_dataset = pd.read_csv('/kaggle/input/titanic-train/titanic_train.csv')
titanic_test_dataset = pd.read_csv('/kaggle/input/titanic-test/titanic_test.csv')

In [3]:
titanic_train_dataset['Sex'] = titanic_train_dataset['Sex'].map({'male': 0, 'female': 1}).fillna(-1)
titanic_train_dataset['Embarked'] = titanic_train_dataset['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).fillna(-1)
titanic_train_dataset['Title'] = titanic_train_dataset['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
titanic_train_dataset['Age'] = titanic_train_dataset.groupby('Title')['Age'].transform(lambda x: x.fillna(x.median()))
titanic_train_dataset['HasCabin'] = titanic_train_dataset['Cabin'].notna().astype(int)
titanic_train_dataset['FamilySize'] = titanic_train_dataset['SibSp'] + titanic_train_dataset['Parch'] + 1
titanic_train_dataset['IsAlone'] = (titanic_train_dataset['FamilySize'] == 1).astype(int)


X = titanic_train_dataset.drop(columns = ['Survived', 'PassengerId', 'Ticket', 'Name', 'Cabin', 'Title'])
Y = titanic_train_dataset['Survived']

In [4]:
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [5]:
titanic_test_dataset['Sex'] = titanic_test_dataset['Sex'].map({'male': 0, 'female': 1}).fillna(-1)
titanic_test_dataset['Embarked'] = titanic_test_dataset['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).fillna(-1)
# Step 1: Combine train + test to calculate age medians
combined = pd.concat([titanic_train_dataset, titanic_test_dataset], sort=False)
title_age_median = combined.groupby('Title')['Age'].median()

# Step 2: Map those medians to test data (after extracting Title)
titanic_test_dataset['Title'] = titanic_test_dataset['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
titanic_test_dataset['Age'] = titanic_test_dataset.apply(
    lambda row: title_age_median[row['Title']] if pd.isna(row['Age']) else row['Age'], axis=1
)

titanic_test_dataset['HasCabin'] = titanic_test_dataset['Cabin'].notna().astype(int)
titanic_test_dataset['FamilySize'] = titanic_test_dataset['SibSp'] + titanic_test_dataset['Parch'] + 1
titanic_test_dataset['IsAlone'] = (titanic_test_dataset['FamilySize'] == 1).astype(int)
titanic_test_dataset['Fare'] = titanic_test_dataset['Fare'].fillna(titanic_test_dataset['Fare'].median())


print("❓ Nulls in X_TEST:", titanic_test_dataset.isnull().sum())

X_TEST = titanic_test_dataset.drop(columns = ['PassengerId', 'Ticket', 'Name', 'Cabin', 'Title'])
print("🔥 Any NaNs before scaling?", pd.DataFrame(X_TEST).isna().sum().sum())
X_TEST = scaler.transform(X_TEST)


❓ Nulls in X_TEST: PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
Title            0
HasCabin         0
FamilySize       0
IsAlone          0
dtype: int64
🔥 Any NaNs before scaling? 0


In [6]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=42)


In [7]:
classifier = svm.SVC(kernel='linear')
classifier.fit(X_train, Y_train)
X_train_prediction = classifier.predict(X_train)
X_val_prediction = classifier.predict(X_val)
print('🔥 Training Accuracy:', accuracy_score(Y_train, X_train_prediction))
print('🧪 Validation Accuracy:', accuracy_score(Y_val, X_val_prediction))

svm_test = classifier.predict(X_TEST)
print("🧊 Test Predictions:", svm_test)
output = pd.DataFrame({
    'PassengerId': titanic_test_dataset['PassengerId'],
    'Survived': svm_test
})
output.to_csv('titanic_svm_test_predictions.csv', index=False)
print("📁 Final Output:")
print(output.head())

🔥 Training Accuracy: 0.7907303370786517
🧪 Validation Accuracy: 0.7877094972067039
🧊 Test Predictions: [0 1 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 1 1 0 0 1 0 1 0 1 0 0 0 0 0 1 1 0 0 1
 1 0 0 0 0 0 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 1 1 0 1 0
 1 0 0 1 0 1 0 0 0 0 0 0 1 1 1 0 1 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 1 1 0 1 1 0 1 0 0 1 0 0 1 1 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1
 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 0 0 1 1 0
 0 1 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 0 1 1 1 1 1 0 1 0 0 0]
📁 Final Output:
   PassengerId  Survived
0       

In [8]:
rf_model = RandomForestClassifier(
    n_estimators=100,  
    max_depth=5,       
    random_state=2
)
rf_model.fit(X_train, Y_train)

X_train_prediction = rf_model.predict(X_train)
X_val_prediction = rf_model.predict(X_val)
print('🔥 Training Accuracy:', accuracy_score(Y_train, X_train_prediction))
print('🧪 Validation Accuracy:', accuracy_score(Y_val, X_val_prediction))

rf_test = rf_model.predict(X_TEST)
print("🧊 Test Predictions:", rf_test)
output = pd.DataFrame({
    'PassengerId': titanic_test_dataset['PassengerId'],
    'Survived': rf_test
})
output.to_csv('titanic_rf_test_predictions.csv', index=False)
print("📁 Final Output:")
print(output.head())

🔥 Training Accuracy: 0.8721910112359551
🧪 Validation Accuracy: 0.7877094972067039
🧊 Test Predictions: [0 0 0 0 0 0 1 0 1 0 0 0 1 0 1 1 0 0 0 1 0 0 1 0 1 0 1 0 0 0 0 0 1 0 1 0 0
 0 0 0 0 1 0 1 1 0 0 0 1 1 1 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 1 1 1 0 0 1
 1 0 0 1 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 1 1 0 1 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 1 1 0 1 1 0 1 1 0 1
 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 1 1 1 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 0 1 0 0 1 0 0 0 1 1 1 1 1 0 0 0 1 0 1 0 1 0 1 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 1 1 0 0 1 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 1 1 1 0 1 0 1 1 0 0 0 1 0 1 0 0 0 0 1 1 0 1 0 0 1 1 0
 0 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 0 1 0 0
 0 1 1 1 1 0 0 1 0 0 1]
📁 Final Output:
   PassengerId  Survived
0       

In [9]:
from tensorflow.keras.callbacks import EarlyStopping
input_dim = X_train.shape[1]
num_classes = len(Y_train.unique())  # 2 in your case
# early_stop = EarlyStopping(patience=10, restore_best_weights=True)
nn_model = Sequential([
    Input(shape=(input_dim,)),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(16, activation='relu'),
    Dense(num_classes, activation='softmax')
])

nn_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = nn_model.fit(X_train, Y_train, epochs=100, batch_size=8, validation_data=(X_val, Y_val))



# 6. Evaluate accuracy
train_acc = history.history['accuracy'][-1]
val_acc = history.history['val_accuracy'][-1]

print(f"🔥 Training Accuracy: {train_acc}")
print(f"🧪 Validation Accuracy: {val_acc}")

nn_test_proba = nn_model.predict(X_TEST)
nn_test = np.argmax(nn_test_proba, axis=1)
print("🧊 Test Predictions:", nn_test)
output = pd.DataFrame({
    'PassengerId': titanic_test_dataset['PassengerId'],
    'Survived': nn_test
})
output.to_csv('titanic_nn_test_predictions.csv', index=False)
print("📁 Final Output:")
print(output.head())

Epoch 1/100


2025-05-24 20:26:01.201568: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


89/89 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.5032 - loss: 0.7147 - val_accuracy: 0.7039 - val_loss: 0.5512
Epoch 2/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7578 - loss: 0.5126 - val_accuracy: 0.7765 - val_loss: 0.4808
Epoch 3/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7816 - loss: 0.4810 - val_accuracy: 0.7989 - val_loss: 0.4541
Epoch 4/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7674 - loss: 0.4826 - val_accuracy: 0.7989 - val_loss: 0.4472
Epoch 5/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8443 - loss: 0.4141 - val_accuracy: 0.7765 - val_loss: 0.4444
Epoch 6/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8434 - loss: 0.3999 - val_accuracy: 0.7989 - val_loss: 0.4384
Epoch 7/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7839 - loss: 0.4755 - val_accuracy: 0.7877 - val_loss: 0.4343
Epoch 8/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8168 - loss: 0.4172 - val_accuracy: 0.7877 - val_loss: 0.4

In [10]:
xgb_model = XGBClassifier(
    n_estimators=50,
    max_depth=4,
    learning_rate=0.05,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=2
)
xgb_model.fit(X_train, Y_train)
X_train_prediction = xgb_model.predict(X_train)
X_val_prediction = xgb_model.predict(X_val)
print('🔥 Training Accuracy:', accuracy_score(Y_train, X_train_prediction))
print('🧪 Validation Accuracy:', accuracy_score(Y_val, X_val_prediction))

xgb_test = xgb_model.predict(X_TEST)
print("🧊 Test Predictions:", xgb_test)
output = pd.DataFrame({
    'PassengerId': titanic_test_dataset['PassengerId'],
    'Survived': xgb_test
})
output.to_csv('titanic_xgb_test_predictions.csv', index=False)
print("📁 Final Output:")
print(output.head())

🔥 Training Accuracy: 0.8609550561797753
🧪 Validation Accuracy: 0.8100558659217877
🧊 Test Predictions: [0 0 0 0 0 0 1 0 1 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 1 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 0 0
 1 0 0 1 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 1 1 1 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1
 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 1 0 1 0 0 0 0 1 1 0 1 0 0 0 1 0
 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 0 1 1 1 1 0 0 1 0 0 1]
📁 Final Output:
   PassengerId  Survived
0       